In [ ]:
# Install dependencies
!pip install pathway bokeh geopy --quiet

In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from time import sleep

In [10]:
parking_df = pd.read_csv('dataset.csv')
parking_df['Timestamp'] = pd.to_datetime(
    parking_df['LastUpdatedDate'] + ' ' + parking_df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

In [11]:
parking_df = parking_df.sort_values(['Timestamp', 'SystemCodeNumber'])

In [13]:
base_price = 10
price_adjustment = 5
demand_sensitivity = 0.5
occ_weight = 0.6
queue_weight = 0.3
traffic_weight = 0.4
special_day_weight = 1.0
vehicle_type_impact = {
    'car': 1,
    'bike': 0.5,
    'truck': 1.5
}
traffic_condition_map = {
    'Low': 0,
    'Average': 1,
    'High': 2
}

def baseline_pricing(current_occupancy, max_capacity, previous_price=base_price):
    if max_capacity == 0:
        return base_price
    raw_price = previous_price + price_adjustment * (current_occupancy / max_capacity)
    return max(5, min(20, round(raw_price, 2)))

def compute_demand(lot_row):
    if lot_row['Capacity']:
        occupancy_ratio = lot_row['Occupancy'] / lot_row['Capacity']
        occupancy_component = occ_weight * occupancy_ratio
    else:
        occupancy_component = 0
    queue_component = queue_weight * lot_row['QueueLength']
    traffic_level = str(lot_row['TrafficConditionNearby']).capitalize()
    traffic_val = traffic_condition_map.get(traffic_level, 1)
    traffic_component = -traffic_weight * traffic_val
    special_day_bonus = special_day_weight * int(lot_row['IsSpecialDay'])
    veh_type = lot_row['VehicleType'].lower()
    vehicle_component = vehicle_type_impact.get(veh_type, 1)
    demand_total = (
        occupancy_component +
        queue_component +
        traffic_component +
        special_day_bonus +
        vehicle_component
    )
    return demand_total

def demand_based_price(demand_val):
    normalized = (demand_val - 1) / 9
    adjusted = base_price * (1 + demand_sensitivity * normalized)
    return max(5, min(20, round(adjusted, 2)))

def get_nearby_lots(all_lots_df, this_lot, radius_km=0.5):
    my_lat = this_lot['Latitude']
    my_lon = this_lot['Longitude']
    nearby_lots = []
    for _, lot in all_lots_df.iterrows():
        if lot['SystemCodeNumber'] == this_lot['SystemCodeNumber']:
            continue
        try:
            dist = geodesic((my_lat, my_lon), (lot['Latitude'], lot['Longitude'])).km
            if dist <= radius_km:
                nearby_lots.append(lot)
        except:
            pass
    return pd.DataFrame(nearby_lots)

def competitive_adjustment(my_price, other_lot_prices):
    if len(other_lot_prices) == 0:
        return my_price
    avg_competitor_price = other_lot_prices['price'].mean()
    if my_price > avg_competitor_price:
        return max(5, my_price - 1)
    elif my_price < avg_competitor_price:
        return min(20, my_price + 1)
    return my_price

In [14]:
data_sources = {}
plot_dict = {}
lot_ids = parking_df['SystemCodeNumber'].unique()

for lot_id in lot_ids:
    data_sources[lot_id] = ColumnDataSource(data={'x': [], 'self_price': [], 'comp_price': []})
    p = figure(
        title=f"Lot {lot_id}: Price vs Competitors",
        x_axis_type="datetime",
        width=600,
        height=300
    )
    p.line(x='x', y='self_price', color='blue', legend_label='Our Price', source=data_sources[lot_id])
    p.line(x='x', y='comp_price', color='red', legend_label='Competitor Avg Price', source=data_sources[lot_id])
    p.legend.location = 'top_left'
    plot_dict[lot_id] = p

layout = column(list(plot_dict.values()))
handle = show(layout, notebook_handle=True)

In [15]:
price_history = {}

for time_point in sorted(parking_df['Timestamp'].unique()):
    snapshot = parking_df[parking_df['Timestamp'] == time_point].copy()
    for idx, row in snapshot.iterrows():
        lot_id = row['SystemCodeNumber']
        prev_price = price_history.get(lot_id, base_price)
        demand = compute_demand(row)
        price = demand_based_price(demand)
        competitors = get_nearby_lots(snapshot, row)
        if not competitors.empty:
            competitors['price'] = competitors.apply(
                lambda r: demand_based_price(compute_demand(r)),
                axis=1
            )
            avg_comp_price = competitors['price'].mean()
            price = competitive_adjustment(price, competitors)
        else:
            avg_comp_price = price
        price_history[lot_id] = price
        data_sources[lot_id].stream({
            'x': [time_point],
            'self_price': [price],
            'comp_price': [avg_comp_price]
        }, rollover=100)
    push_notebook(handle=handle)
    sleep(0.2)

/var/folders/2_/qqds09zj3gb53csskrxv_wgh0000gn/T/ipykernel_34383/1340908917.py:26: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  push_notebook(handle=handle)
/var/folders/2_/qqds09zj3gb53csskrxv_wgh0000gn/T/ipykernel_34383/1340908917.py:26: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  push_notebook(handle=handle)
/var/folders/2_/qqds09zj3gb53csskrxv_wgh0000gn/T/ipykernel_34383/1340908917.py:26: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  push_notebook(handle=handle)
/var/folders/2_/qqds09zj3gb53csskrxv_wgh0000gn/T/ipykernel_34383/1340908917.py:26: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  push_notebook(handle=handle)
